# Analysis

In [ ]:
import pandas as pd

In [ ]:
input_csv = "../data/plos2016.csv"
urls_csv = "../data/urls.csv"
query_csv = "../data/queries_12k.csv"
og_csv = "../data/og_objects_12k.csv"

In [ ]:
articles = pd.read_csv(input_csv, index_col="doi")
urls = pd.read_csv(urls_csv, index_col="url_id")
queries = pd.read_csv(query_csv, index_col="query_id")
og_objects = pd.read_csv(og_csv, index_col="og_id")

In [ ]:
og_objects[["reactions", "shares", "comments", "plugin_comments"]] = og_objects[["reactions", "shares", "comments", "plugin_comments"]].astype(int)

In [ ]:
a = og_objects.merge(queries, left_on="query_id", right_index=True, how="left")
b = a.merge(urls, left_on="url_id", right_index=True, how="left")
c = b.merge(articles, left_on="doi", right_index=True, how="left")

In [ ]:
dupl_queries = og_objects[og_objects.index.duplicated(keep=False)].query_id

In [ ]:
c[c.query_id.isin(dupl_queries)].groupby("type")[['reactions', 'shares', 'comments', 'plugin_comments']].mean()

In [ ]:
x = c.reset_index().groupby(["doi", "og_type"])['og_id'].apply(lambda x: len(set(x))!=1)
x[x]

In [ ]:
c[c.doi.isin(xdois)].groupby("doi")['shares'].count()

In [ ]:
a = a[~a.query_id.isin(dupl_queries)]
b = b[~b.query_id.isin(dupl_queries)]
c = c[~c.query_id.isin(dupl_queries)]

In [ ]:
mean = b.groupby("type")[['shares']].mean()
count = b.groupby("type")[['shares']].count()

mean['size'] = b.groupby("type").size()
mean

In [ ]:
b.groupby("type")[['reactions', 'shares', 'comments', 'plugin_comments']].mean()

In [ ]:
# Total shares by article
c.groupby(["doi"])[['shares']].sum().sort_values("shares", ascending=False).head()

In [ ]:
c.columns

In [ ]:
c[c.doi=="10.1371/journal.pone.0156632"].groupby("type").size()

In [ ]:
og_objects[["reactions", "shares", "comments", "plugin_comments"]] = og_objects[["reactions", "shares", "comments", "plugin_comments"]].astype(int)